In [8]:
import requests
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import pickle
import copy
import os
import math

pd.set_option('display.max_rows', 500)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline  
import seaborn as sns
sns.set_style("darkgrid")

import umap
from sklearn.decomposition import TruncatedSVD as tsvd

def nearZeroVarDropAuto(df,thresh=0.99):
    vVal=df.var(axis=0).values
    cs=pd.Series(vVal).sort_values(ascending=False).cumsum()
    remove=cs[cs>cs.values[-1]*thresh].index.values
    return df.drop(df.columns[remove],axis=1)

%run SodaKick_download_functions.ipynb


import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.multioutput import MultiOutputRegressor

import xgboost as xgb
from xgboost import XGBRegressor

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.xgboost import TuneReportCheckpointCallback
from functools import partial 
from ray.tune.suggest.hyperopt import HyperOptSearch

from sklearn.model_selection import cross_val_score
#https://docs.ray.io/en/master/tune/tutorials/tune-xgboost.html

import torch

In [9]:
#from ray import tune
#from ray.tune import CLIReporter
#from ray.tune.schedulers import ASHAScheduler

from hyperopt import hp, tpe, fmin, Trials
from hyperopt import STATUS_OK, STATUS_FAIL

In [10]:
#https://towardsdatascience.com/quirky-keras-custom-and-asymmetric-loss-functions-for-keras-in-r-a8b5271171fe
#weighted asimmetric square error, errors by going below the value (not seeing a goal when it's there) are weighted more

def WSE(output, target, a=1.5, b=.5):
    loss = torch.mean(a/(a+b)*torch.minimum(torch.zeros(output.shape[1]),output - target)**2+\
                      b/(a+b)*torch.maximum(torch.zeros(output.shape[1]),output - target)**2)      
    return loss

def WSEl1(output, target, a=1.5, b=.5):
    loss = torch.mean(a/(a+b)*torch.abs(torch.minimum(torch.zeros(output.shape[1]),output - target))+\
                      b/(a+b)*torch.abs(torch.maximum(torch.zeros(output.shape[1]),output - target)))      
    return loss

def WSE2(output, target, a=1.5, b=.5):
    loss = np.mean(a/(a+b)*np.minimum(np.zeros(output.shape[0]),output - target)**2+\
                      b/(a+b)*np.maximum(np.zeros(output.shape[0]),output - target)**2)      
    return loss

def WSEl12(output, target, a=1.5, b=.5):
    loss = np.mean(a/(a+b)*np.abs(np.minimum(np.zeros(output.shape[0]),output - target))+\
                      b/(a+b)*np.abs(np.maximum(np.zeros(output.shape[0]),output - target)))      
    return loss

def log_cosh_loss(y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
    def _log_cosh(x: torch.Tensor) -> torch.Tensor:
        return x + torch.nn.functional.softplus(-2. * x) - math.log(2.0)
    return torch.mean(_log_cosh(y_pred - y_true))

class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(
        self, y_pred: torch.Tensor, y_true: torch.Tensor
    ) -> torch.Tensor:
        return log_cosh_loss(y_pred, y_true)

In [13]:
from typing import Tuple

def WSE(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    
    target = dtrain.get_label()
    predt[predt < -1] = -1 + 1e-6
    
    a=1.5
    b=.5
    
    elements = a*np.minimum(np.zeros(len(predt)),predt - target)**2+\
                      b*np.maximum(np.zeros(len(predt)),predt - target)**2
    
    return 'WSE', float(np.sqrt(np.sum(elements) / len(target)))

In [11]:
def normalize_mins(vec):
    for i in range(vec.shape[0]):
        vec[i][::8]=vec[i][::8]/90

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def NormalizeMatrix(data):   
    for i in range(data.shape[1]):
        data[:,i] = NormalizeData(data[:,i])

def norm_max(out):
    
    maxes=[]
    for i in range(int(out.shape[1]/8.0)):
        maxes.append(out[:,8*int(i):8*(int(i)+1)].max(axis=0))

        #maxes.append(out.max(axis=1)[8*int(i):8*(int(i)+1):8])
    denominator=np.tile(np.max(maxes,axis=0),int(out.shape[1]/8))
    return out/denominator, denominator 

with open(r'/Users/federico comitani/GitHub/sodakick/data/wainp_220303.pkl', 'rb') as pk:
    inp=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/out_220303.pkl', 'rb') as pk:
    out=np.array(pickle.load(pk),dtype=float)
    
### skipping norm for now since it's already tsvd 
#NormalizeMatrix(inp)
#np.nan_to_num(inp, copy=False)

from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()
inp = scaler.fit_transform(inp)

#normalize_mins(out)
out, denominator= norm_max(out)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
         inp[:25000], out[:25000], test_size=0.2, random_state=32)

In [49]:
def train_xgb(config, col=0):
    
    #try:
        
        model = XGBRegressor(**config)
        eval_set = [(x_train, y_train[:,col]),(x_test, y_test[:,col])]
        model.fit(x_train, y_train[:,col], eval_metric=config['eval_metric'], eval_set=eval_set, early_stopping_rounds = 50, verbose=False)    

        evals_result = model.evals_result()

        return {'loss': evals_result['validation_1'][config['eval_metric']][-1], 'status': STATUS_OK}
    
    #except:
        
    #    return {'loss': np.nan, 'status': STATUS_FAIL}

In [50]:
def revert_output(output,multiplier=denominator,lineup=None):

    reframe=pd.DataFrame(output.reshape(48,8),
                 columns=['minutes','goals','assists','cards_yellow','cards_red','own_goals','goals_against','saves'])
    
    reframe[reframe<0] = 0
    if lineup is not None:
        reframe.index=lineup
        reframe.drop([x for x in reframe.index if x.startswith('dummy')], axis=0, inplace=True)
    
    
    #reframe['minutes']*=90
    reframe=reframe*denominator[:8]
    byteamframe=pd.concat([reframe.iloc[:24,:].sum(axis=0),reframe.iloc[24:,:].sum(axis=0)], axis=1).T
    
    return reframe, byteamframe[byteamframe.columns[1:]]

In [51]:
print('Baseline WSE: {:.3f}'.format(WSE2(np.array([0]*out[0].shape[0]),out[0])))
print('Baseline WSE l1: {:.3f}'.format(WSEl12(np.array([0]*out[0].shape[0]),out[0])))
print('Baseline MSE: {:.3f}'.format(WSE2(np.array([0]*out[0].shape[0]),out[0], a=1, b=1)))
print('Baseline MSE l1: {:.3f}'.format(WSEl12(np.array([0]*out[0].shape[0]),out[0], a=1, b=1)))

print(np.abs(out[1]-out[10]).sum())
print(np.abs(out[50]-out[60]).sum())
print(np.abs(out[100]-out[110]).sum())

Baseline WSE: 0.039
Baseline WSE l1: 0.052
Baseline MSE: 0.026
Baseline MSE l1: 0.035
36.36507936507937
24.09365079365079
34.76825396825397


In [52]:
def run_hopt(config, num_samples=10, col=0):#, gpus_per_trial=2):
    
    trials = Trials()
    result = fmin(
            fn=partial(train_xgb, col=col),
            #train_xgb,
            space=config,
            algo=tpe.suggest,
            max_evals=num_samples,
            trials=trials,
            show_progressbar=True),
            #early_stop_fn=10,
            #trials_save_file=None)
    
    
    return trials
    #return best_trained_model
    #test_acc = test_accuracy(best_trained_model, device)
    #print("Best trial test set accuracy: {}".format(test_acc))

In [63]:
search_space = {
 "n_estimators": 25,
 "max_depth": 2+hp.randint('max_depth', 13),
 "min_child_weight": hp.choice("min_child_weight",[1, 2, 3, 4, 5]),
 "subsample": hp.choice("subsample",np.linspace(.5,.9,5)),
 "eta": hp.choice("eta",[1e-2, 5e-2, 1e-1, 5e-1, 3e-1]),
 "colsample_bytree": hp.choice("colsample_bytree",np.linspace(0.1,.9,9)),
 "alpha": hp.randint("alpha", 5),
 "lambda": hp.randint("lambda", 10),
 "gamma" : hp.choice("gamma",np.linspace(0,.9,10)),
 "objective": "reg:pseudohubererror",
 "eval_metric": "rmse", 
 "learning_rate": 1e-1, 
 }


allres=[]
for i in range(8):
    print('Feature #: '+str(i))
    btm = run_hopt(search_space, num_samples=50, col=i)
    
    results_df=[]
    for trial in btm.trials:
        results_df.append([trial['result']['loss'],
        int(trial['misc']['vals']["max_depth"][0]),
        [1, 2, 3, 4, 5][trial['misc']['vals']["min_child_weight"][0]],
        np.linspace(.5,.9,5)[trial['misc']['vals']["subsample"][0]],
        [1e-2, 5e-2, 1e-1, 5e-1, 3e-1][trial['misc']['vals']["eta"][0]],
        np.linspace(0.1,.9,9)[trial['misc']['vals']["colsample_bytree"][0]],
        trial['misc']['vals']["alpha"][0],
        trial['misc']['vals']["lambda"][0],
        np.linspace(0,.9,10)[trial['misc']['vals']["gamma"][0]],                 
        ])

    allres.append(pd.DataFrame(results_df,columns=['loss',
                                            'max_depth',
                                            'min_child_weight',
                                            'subsample',
                                            'eta',
                                            'colsample_bytree',
                                            'alpha',
                                            'lambda',
                                            'gamma',
                                            ]).sort_values('loss').iloc[0])
    allres[-1].name=i
    
allres=pd.concat(allres,axis=1).T

Feature #: 0
100%|██████████| 50/50 [07:36<00:00,  9.12s/trial, best loss: 0.136745]
Feature #: 1
100%|██████████| 50/50 [03:54<00:00,  4.69s/trial, best loss: 0.06655] 
Feature #: 2
100%|██████████| 50/50 [03:35<00:00,  4.31s/trial, best loss: 0.057487]
Feature #: 3
100%|██████████| 50/50 [04:22<00:00,  5.25s/trial, best loss: 0.133172]
Feature #: 4
100%|██████████| 50/50 [03:35<00:00,  4.31s/trial, best loss: 0.06808] 
Feature #: 5
100%|██████████| 50/50 [02:45<00:00,  3.31s/trial, best loss: 0.055893]
Feature #: 6
100%|██████████| 50/50 [03:45<00:00,  4.51s/trial, best loss: 0.048276]
Feature #: 7
100%|██████████| 50/50 [03:44<00:00,  4.49s/trial, best loss: 0.049079]


In [64]:
best_cfg=[]
for i in range(8):
    best_cfg.append(allres.iloc[i].to_dict())
    best_cfg[-1]["objective"]="reg:pseudohubererror"
    best_cfg[-1]["eval_metric"]="rmse"
    best_cfg[-1]["learning_rate"]=1e-1
    best_cfg[-1]["n_estimators"]=100

In [65]:
from sklearn.model_selection import KFold

def runKfold(indata, outdata, num, best_cfg=best_cfg):
    
    kf = KFold(n_splits=5, shuffle=True)
    kf.get_n_splits(indata)
    
    tmpparms = best_cfg[num]
    tmpparms.pop("file", None)    
    tmpparms['n_estimators']=1000
    
    print(tmpparms)
        
    ec,losses, errors = [], [], []
    rmses = []
    results=[]
    
    for train_index, test_index in kf.split(indata):        
        x_train, y_train, x_test, y_test = indata[train_index], outdata[train_index], indata[test_index], outdata[test_index]

        train_set = xgb.DMatrix(x_train, label=y_train[:,i])
        test_set = xgb.DMatrix(x_test, label=y_test[:,i])
    
        model=XGBRegressor(**tmpparms)
        model.fit(x_train, y_train[:,i],
            eval_set = [(x_train, y_train[:,i]),(x_test, y_test[:,i])],
            eval_metric = 'rmse',
            early_stopping_rounds = 10, verbose=False)
        
        results.append(model.evals_result())
        best_iteration = model.get_booster().best_ntree_limit
        
        #pred = model.predict(x_test, ntree_limit=best_iteration)
        #errors.append(np.mean(pred-y_test[:,i]))

        ec.append(model.best_iteration)
        losses.append(model.get_booster().best_score)
        #rmses.append(results[-1]['validation_1']['rmse'][-1])
    
    print('Num: {:.3f}+/-{:.3f}'.format(np.mean(ec),np.std(ec)))
    print('KFold Result: {:.3f}+/-{:.3f}'.format(np.mean([np.mean(x) for x in losses]),np.std([np.mean(x) for x in losses])))
    #print('Error Result: {:.3f}+/-{:.3f}'.format(np.mean([np.mean(x) for x in errors]),np.std([np.mean(x) for x in errors])))
    #print('RMSE Result: {:.3f}+/-{:.3f}'.format(np.mean(rmses),np.std(rmses)))

    # plot log loss
    fig, ax = plt.subplots()
    for rs in results:
        ax.plot(range(len(rs['validation_0']['rmse'])), rs['validation_0']['rmse'], label='Train', color='blue')
        ax.plot(range(len(rs['validation_1']['rmse'])), rs['validation_1']['rmse'], label='Test', color='orange')
    plt.show()
    
    return np.mean([np.mean(x) for x in losses]), np.mean(ec)

In [66]:
rmses, epochss=[],[]
for i in range(8):
    print('Feature #: '+str(i))
    rmse, epochs = runKfold(inp, out, i, best_cfg)
    rmses.append(rmse)
    epochss.append(epochs)
    print()

Feature #: 0
{'loss': 0.136745, 'max_depth': 11.0, 'min_child_weight': 5.0, 'subsample': 0.6, 'eta': 0.5, 'colsample_bytree': 0.6, 'alpha': 0.0, 'lambda': 4.0, 'gamma': 0.4, 'objective': 'reg:pseudohubererror', 'eval_metric': 'rmse', 'learning_rate': 0.1, 'n_estimators': 1000}


XGBoostError: Invalid Parameter format for max_depth expect int but value='11.0'